# Importo librerías.

In [ ]:
!pip install Faker pandas;


# Genero la BD SQLite con datos ficticios.

In [ ]:
import sqlite3
from faker import Faker
import random

# Conectar a la base de datos (crea el archivo si no existe)
conn = sqlite3.connect('datos.db')
cursor = conn.cursor()

cursor.executescript('''
DROP TABLE IF EXISTS empleado;
CREATE TABLE empleado (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pais TEXT NOT NULL,
    nombre TEXT NOT NULL,
    edad INTEGER,
    puesto TEXT,
    activo INTEGER CHECK(activo IN (0, 1)),
    sueldo REAL
);
''')

fake = Faker('es_ES') # Generar datos en español

for _ in range(10000):
    pais = fake.country()
    nombre = fake.name()
    edad = random.randint(22, 65)
    puesto = fake.job()
    activo = random.choice([0, 1])
    sueldo = round(random.uniform(25000.00, 150000.00), 2)

    cursor.execute('''
    INSERT INTO empleado (pais, nombre, edad, puesto, activo, sueldo)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (pais, nombre, edad, puesto, activo, sueldo))

cursor.executescript('''
DROP TABLE IF EXISTS modelo_auto;
CREATE TABLE modelo_auto (
    modelo_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_modelo TEXT NOT NULL
);
''')

modelos = ["Sedan", "SUV", "Hatchback", "Pickup", "Coupe"]
for modelo in modelos:
    cursor.execute("INSERT INTO modelo_auto (nombre_modelo) VALUES (?)", (modelo,))

cursor.executescript('''
DROP TABLE IF EXISTS auto_vendido;
CREATE TABLE auto_vendido (
    venta_id INTEGER PRIMARY KEY AUTOINCREMENT,
    modelo_id INTEGER NOT NULL,
    fecha_venta DATE,
    precio REAL,
    FOREIGN KEY (modelo_id) REFERENCES modelo_auto(modelo_id)
);
''')

fake = Faker('es_ES')

for _ in range(1000):
    modelo_id = random.randint(1, len(modelos))
    fecha_venta = fake.date_between(start_date='-2y', end_date='today')
    precio = round(random.uniform(10000.00, 80000.00), 2)

    cursor.execute('''
    INSERT INTO auto_vendido (modelo_id, fecha_venta, precio)
    VALUES (?, ?, ?)
    ''', (modelo_id, fecha_venta, precio))


conn.commit()

# Tablas

| Empleado | Modelo_Auto | Auto_Vendido |
|---|---|---|
|ID|ID|venta_id|
|Pais|Nombre_Modelo|modelo_id|
|Nombre||fecha_venta|
|Edad||precio|
|Puesto|
|Activo|
|Sueldo|



# Consultas

In [ ]:
# @title Select {"run":"auto","form-width":"1000px"}
import pandas as pd
conn = sqlite3.connect('datos.db')
consulta = "select * from empleado;" # @param {"type":"string","placeholder":"select * from empleado;"}
df = pd.read_sql_query(consulta, conn)
conn.close()

df

In [ ]:
# @title Consulta con Algo Mas {"run":"auto","form-width":"1000px"}
import sqlite3
import pandas as pd

conn = sqlite3.connect('datos.db')

consulta = "select nombre,edad,sueldo from empleado where activo = 1 and pais = 'Argentina' order by edad desc limit 10;" # @param {"type":"string","placeholder":"select * from empleados;"}

df = pd.read_sql_query(consulta, conn)
conn.close()

df

---

Ver orden de ejecución en Presentación

---

# Joins

In [ ]:
# @title Ejemplo Joins {"run":"none","form-width":"1000px"}
conn = sqlite3.connect('datos.db')

consulta = "select vendido.modelo_id, fecha_venta, precio, modelo.nombre_modelo from auto_vendido vendido left join modelo_auto modelo on vendido.modelo_id = modelo.modelo_id;" # @param {"type":"string","placeholder":"sselect vendido.modelo_id, fecha_venta, precio, modelo.nombre_modelo from auto_vendido vendido left join modelo_auto modelo on vendido.modelo_id = modelo.modelo_id;"}

df = pd.read_sql_query(consulta, conn)
conn.close()

df

---

Ver Joins en Presentación

---

# Análisis

In [ ]:
# @title Análisis - Top 10 Plantilla mas Veterana {"run":"auto","form-width":"1000px"}
conn = sqlite3.connect('datos.db')

query = "select pais, count(*) q_empleados, round(avg(edad),2) prom_edad from empleado group by pais order by avg(edad) desc limit 10; " # @param {"type":"string","placeholder":"select * from empleados;"}

df = pd.read_sql_query(query, conn)
conn.close()

df

In [ ]:
# @title Análisis - Top 3 Sueldos Por Pais {"run":"auto","form-width":"1000px"}
conn = sqlite3.connect('datos.db')

query = "select * from (select pais, id,nombre, sueldo, rank() over (partition by pais order by sueldo desc) ranking from empleado where activo = 1) where ranking < 4 order by pais limit 100;" # @param {"type":"string","placeholder":"select * from empleados;"}

df = pd.read_sql_query(query, conn)
conn.close()

df